In [4]:
import os,sys
import kfp
import json

In [5]:
components_url = "/mnt/dkube/pipeline/components/"
dkube_preprocessing_op   = kfp.components.load_component_from_file(components_url + "preprocess/component.yaml")
dkube_training_op        = kfp.components.load_component_from_file(components_url + "training/component.yaml")
dkube_serving_op         = kfp.components.load_component_from_file(components_url + "serving/component.yaml")
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(host=os.getenv("KF_PIPELINES_ENDPOINT"), existing_token=token, namespace=os.getenv("USERNAME"))

In [23]:
preprocessing_script = f"pip install py7zr --user && python cifar10preprocessing.py"
input_mount_point = "/opt/dkube/input"
output_mount_point = "/opt/dkube/output"

In [31]:
@kfp.dsl.pipeline(
    name='dkube-mnist-pl',
    description='sample mnist pipeline with dkube components'
)
def cifar10_pipeline(program='cifar10', dataset_preprocess='cifar10', dataset='cifar10-extracted', model='cifar10'):
#     print(dataset_preprocess)
    
    preprocessing = dkube_preprocessing_op(auth_token = token,container=json.dumps({"image": "ocdr/dkube-datascience-tf-cpu:v2.0.0"}),
                                           program=str(program), run_script=preprocessing_script,
                                           datasets=json.dumps([str(dataset_preprocess)]), 
                                           outputs=json.dumps([str(dataset)]),
                                           input_dataset_mounts=json.dumps([input_mount_point]), 
                                           output_mounts=json.dumps([output_mount_point])
                                            )

    train        = dkube_training_op(container='{"image":"ocdr/dkube-datascience-tf-cpu:v2.0.0"}',
                                    framework="tensorflow", version="2.0.0",
                                    program=str(program), run_script="python cifar10training.py",
                                    datasets=json.dumps([str(dataset)]), outputs=json.dumps([str(model)]),
                                    input_dataset_mounts=json.dumps([input_mount_point]),
#                                     input_dataset_mounts='["/cifar10"]',
                                    output_mounts=json.dumps([output_mount_point]),
#                                     output_mounts='["/model"]',
                                    envs='[{"EPOCHS": "1"}]',
                                    auth_token=token).after(preprocessing)

    serving     = dkube_serving_op(model=train.outputs['artifact'], device='cpu', 
                                    serving_image='{"image":"ocdr/tensorflowserver:2.0.0"}',
                                    auth_token=token).after(train)

In [27]:
#     serving     = dkube_serving_op(model=train.outputs['artifact'], device='cpu', 
#                                     serving_image='{"image":"ocdr/tensorflowserver:2.0.0"}',
#                                     transformer_image='{"image":"ocdr/dkube-datascience-tf-cpu:v2.0.0"}',
#                                     transformer_project=str(program),
#                                     transformer_code='mnist/transformer.py', auth_token=token).after(train)

In [32]:
client.create_run_from_pipeline_func(cifar10_pipeline, arguments={})

RunPipelineResult(run_id=312b0642-8779-466b-bc98-69706942e1d9)

In [ ]:
#generate & upload pipeline (Optional)
import kfp.compiler as compiler
compiler.Compiler().compile(cifar10_pipeline, "cifar10-pipeline.zip")
client.upload_pipeline("cifar10-pipeline.zip")